# RSS: Expansión

## 1. Import Libraries

In [11]:
import feedparser
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import googletrans
import datetime
from termcolor import colored
from IPython.display import display, Markdown, Latex, HTML
import re
from IPython.display import Image

In [12]:
def get_only_text(url):
    
    """
    return the title and the text of the article at the 
    specified url
    """
    
    page=urlopen(url)
    soup=BeautifulSoup(page,"html.parser")
    text=' '.join(map(lambda p: p.text, soup.find_all('p')))
    # Eliminar Última Hora
    rest = soup.find_all('p', attrs={'class' : "line-clamp_x2 ue-c-widget__article-headline"})[0].text
    index = text.find(rest)
    final_text = text[:index]
    return [final_text,soup.title.text]

def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""

In [13]:
def expansion_news(expansion_entries,word_count=100):
    expansion = {}
    for i in range(len(expansion_entries)):
        noticia = {'newspaper':None,'title':None,'headline':None,'summarise':None,'date':None,
                   'link':None,'text':None,'img':None}
        # Periodico
        noticia['newspaper'] = find_between(s=expansion_entries[i]['id'], first='www.', last='.' )
        # Title
        noticia['title'] = expansion_entries[i]['title']
        # Headline
        index = expansion_entries[i]['summary_detail']['value'].find('&nbsp')
        if index != -1:
            noticia['headline'] = expansion_entries[i]['summary_detail']['value'][:index]
        else:
            noticia['headline'] = re.sub(pattern='&quot;',repl='"',string=expansion_entries[i]['summary_detail']['value'])
        # Date
        noticia['date'] = {'date1':expansion_entries[i]['published'],'date2':expansion_entries[i]['published_parsed']}
        # Link
        noticia['link'] = expansion_entries[i]['id']
        # Text
        text = get_only_text(expansion_entries[i]['id'])[0]
        noticia['text'] = text
        # Summarise
        if text.find('Para seguir leyendo') == -1:
            noticia['summarise'] = summarize(text, word_count=word_count)
        else:
            noticia['summarise'] = 'Premium Content'
        # Image
        noticia['img'] = expansion_entries[i]['media_content'][0]['url']
        expansion.update({'Noticia'+'_'+str("%02d")%(i+1):noticia})
    return expansion

## 2. RSS Expansión

In [14]:
url_expansion = 'https://e00-expansion.uecdn.es/rss/empresasbanca.xml'
expansion = feedparser.parse(url_expansion)
expansion_entries = expansion['entries']

print('Total Entradas Expansión:',len(expansion_entries))

Total Entradas Expansión: 35


In [15]:
expansion = expansion_news(expansion_entries,word_count=100)

IndexError: list index out of range

In [ ]:
yesterday = datetime.datetime.today() - datetime.timedelta(days=1)
before_yes = datetime.datetime.today() - datetime.timedelta(days=2)

for i in list(expansion.keys()):
    if expansion[i]['date']['date2'][2] in [datetime.datetime.today().day,yesterday.day,before_yes.day]:
        print(colored((expansion[i]['newspaper']).upper(),'green',attrs=['bold']))
        print(colored(expansion[i]['date']['date1'],'grey',attrs=['bold']))
        print(colored(expansion[i]['title'],'magenta',attrs=['bold']))
        print(colored('Headline','blue'))
        print(expansion[i]['headline'])
        print(colored('Summarise','blue'))
        
        if expansion[i]['summarise'] != 'Premium Content':
            print(expansion[i]['summarise'])
        else:
            print(colored(expansion[i]['summarise'],'red'))
        print(colored('Text','blue'))
        print(expansion[i]['text'])
        url = expansion[i]['img']
        display(Image(url=url,width=500))
        display(Markdown('<a href="'+expansion[i]['link']+'">Link</a>'))
        print('\n')